<a href="https://colab.research.google.com/github/thetinybug/MLDL-LearnPyTorch/blob/master/Torch-Grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grad trong PyTorch**
Grad trong PyTorch có thể hiểu là dùng để tính giá trị đạo hàm.  
Các ví dụ dưới đây sẽ làm rõ hơn về chức năng của grad

## **1. Ví dụ 1**

In [23]:
# lets make basic backward propagation
# we have an equation that is y = x^2
array = [2,4]
tensor = torch.Tensor(array)
x = Variable(tensor, requires_grad = True)
y = x**2
# y = Variable(y, requires_grad = True)          # (1)
print(" y =  ",y)

# recap o equation o = 1/2*sum(y)
o = (1/2)*sum(y)
print(" o =  ",o)

# backward
o.backward() # calculates gradients

# As I defined, variables accumulates gradients. In this part there is only one variable x.
# Therefore variable x should be have gradients
# Lets look at gradients with x.grad
print("gradients: ",x.grad)                   #(2)
# print("gradients: ",y.grad)                   #(3)

 y =   tensor([ 4., 16.], grad_fn=<PowBackward0>)
 o =   tensor(10., grad_fn=<MulBackward0>)
gradients:  tensor([2., 4.])


### **NHẬN XÉT**

* Chúng ta có thể thấy giá trị thu được là [2,4] chính là x. Do đạo hàm của o theo x = x.  
*    Cùng ví dụ này, để hiểu rõ hơn, chúng ta có thể tính đạo hàm của o theo y bằng cách: Bỏ comment ***(1), (3)*** và comment ***(2)***






##**2. Ví dụ 2**

In [21]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(f"Loss of Epoch %d: \t%f3"%(t, loss.item()))

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        # Xóa giá trị grad bằng cách gán = 0.
        w1.grad.zero_()
        w2.grad.zero_()


Loss of Epoch 99: 	1047.2076423
Loss of Epoch 199: 	10.9226693
Loss of Epoch 299: 	0.1931073
Loss of Epoch 399: 	0.0044343
Loss of Epoch 499: 	0.0003043


###**Nhận xét**


* w1.grad, w2.grad là các đạo hàm tương ứng với w1, w2 của hàm loss.
* w1.grad.zero_() là để xóa bỏ giá trị grad (make sure không ảnh hưởng tới lần sau. Chi tiết tham khảo câu trả lời của [samarth-robo](https://discuss.pytorch.org/t/why-do-we-need-to-set-the-gradients-manually-to-zero-in-pytorch/4903/3)). Bằng cách gán tất cả giá trị thành 0.

